In [3]:
import sys
sys.path.append('../..')
import pandas as pd
import src.cleaning as clean
import seaborn as sns
import src.analysis.utils as utils
import numpy as np

In [4]:
sales_data = pd.read_csv('../../data/anon_data.csv')
cleaned_data = clean.clean_data(sales_data)

C:\Users\C\AppData\Local\Temp\ipykernel_18448\1800904834.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  sales_data = pd.read_csv('../../data/anon_data.csv')


In [5]:
#Restrict to only orders being sent not returned - being quantity > 0 
positive_quantity = cleaned_data.loc[cleaned_data['quantity']>0, :]

In [6]:
customer_shipped_dates = positive_quantity.groupby('customer_id')['ship_date'].unique()

In [9]:
customer_shipped_days_between = customer_shipped_dates.apply(utils.calculate_days_between)
customer_shipped_days_between

customer_id
100000003                                  [512, 6, 20, 8, 61]
100000005                                          [547, 1, 1]
100000015                                        [82, 372, 19]
100000029                                                 None
100000034    [1, 5, 2, 5, 11, 10, 6, 124, 7, 19, 13, 2, 39,...
                                   ...                        
100024529                                                  [3]
100024593                                                 None
100024597                                                  [4]
100024613                                                 None
100024630                                                 None
Name: ship_date, Length: 3288, dtype: object

In [10]:
customer_shipped_dates.apply(len).sum()

86198

In [6]:
average_customer_shipping_times = customer_shipped_days_between.dropna().apply(np.mean).sort_values()
average_customer_shipping_times[(average_customer_shipping_times > 0) & (average_customer_shipping_times <= 2)].shape

(60,)

In [16]:
shipping_difference_equal_1 = customer_shipped_days_between.apply(lambda row: utils.calculate_result_of_waiting(row, max_days_to_wait=1))
shipping_difference_equal_1.sum()

20036

In [15]:
# CHECKING
# If there is one shipment per customer as we wait for all their orders, this should be equal to number of customers
# Number of shipments - Number to subtract if we wait for all orders == Number of customers
customer_shipped_dates.apply(len).sum() - customer_shipped_days_between.apply(lambda row: utils.calculate_result_of_waiting(row, max_days_to_wait=12000)).sum() == positive_quantity['customer_id'].unique().shape[0]

True

In [19]:
# Additional Checks
customer_shipped_dates[100000354]

<DatetimeArray>
['2021-04-11 00:00:00', '2021-04-21 00:00:00', '2021-04-23 00:00:00',
 '2021-04-27 00:00:00', '2021-05-05 00:00:00', '2021-05-06 00:00:00',
 '2021-05-13 00:00:00', '2021-05-26 00:00:00', '2021-05-27 00:00:00',
 '2021-06-23 00:00:00', '2021-07-12 00:00:00', '2021-07-20 00:00:00',
 '2021-09-09 00:00:00', '2021-10-26 00:00:00', '2021-10-28 00:00:00',
 '2022-06-01 00:00:00', '2022-06-13 00:00:00', '2022-06-15 00:00:00',
 '2022-06-16 00:00:00', '2022-07-20 00:00:00', '2022-07-27 00:00:00',
 '2022-08-18 00:00:00', '2022-08-22 00:00:00', '2022-08-31 00:00:00',
 '2022-10-04 00:00:00', '2022-10-20 00:00:00', '2022-11-02 00:00:00',
 '2022-11-03 00:00:00', '2022-11-21 00:00:00', '2022-12-08 00:00:00',
 '2022-12-09 00:00:00', '2022-12-15 00:00:00', '2023-01-09 00:00:00',
 '2023-01-13 00:00:00', '2023-03-09 00:00:00']
Length: 35, dtype: datetime64[ns]

In [11]:
86198 - 20036

66162